## RouterAgent

智能体路由可以扩展为更广泛的场景，比如根据不同的工具集合、向量库类型等，配置为不同的智能体，
然后根据用户的意图和场景，使用 RouterAgent 动态选择最佳的智能体。

RouterAgent 有两个参数：
- condition 选择函数，可以是普通函数、 lambda 函数，或者实现了 `__call__` 方法的对象
    - condition 接受两个参数，一是提示语，二是智能体名称列表
- agents 智能体列表

### 基础示范

In [3]:
from illufly.types import RouterAgent
from illufly.chat import ChatQwen, ChatZhipu, FakeLLM

def myfunc(agents, consumer_dict):
    return agents[0]

agent = RouterAgent(myfunc, [FakeLLM(), ChatQwen(), ChatZhipu()])
agent.bind_provider({"task": "写一首歌"})

agent("你是什么模型？")

这是一个模拟调用!

'这是一个模拟调用!'

In [4]:
agent.selected

<FakeLLM.4425535632>

### 路由 ChatAgent

考虑到你有两个专业的 ChatAgent:

- data_agent 擅长数据分析
- writer_agent 擅长写作

你已经根据为他们分别配置了提示语模板、工具集、数据库等，但现在你需要根据用户的意图匹配到底使用哪一个智能体。<br>

那么如何判断用户意图呢？
- 方案1 可以使用大模型来做甄别，这会增加一个环节，推理过程可能延长用户的等待
- 方案2 可以根据问题做文本相似性比较，这样做效率很高

我们的例子中探讨方案2如何实现，关键就是与什么做比较？

原始的可比较信息是：用户问题与智能体描述文本之间的比较，但这明显不太够用。<br>
可以使用**检索器**和**经验缓存**来弥补。


In [8]:
agent.consumer_dict

{'task': '写一首歌'}

### 路由 Template

In [1]:
from illufly.types import Template, RouterAgent

r = RouterAgent(
    condition=lambda runs, vars: runs[0],
    runnables=[Template(text="请帮我：{{task}}"), Template(text="你是一个作家")]
)
r.bind_provider({"task": "写一首歌"})

r()

'请帮我：写一首歌'

In [2]:
r.selected

<Template consumer_dict={'task'} text='请帮我：{{task}}'>

### 路由 Retrieval

### 路由 VectorDB